In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from project_utils import get_general_instructions_task, get_condition_instructions_task, get_consent_task, get_experience_rating

In [3]:
def get_segment_tasks(fpath, name, id, recognition_instructions, rating_instructions, instructions_type, max_sub=2):
    
    return [
      {
            "type": "Continuous1DTask",
            "name": f'{name}: Recognition' if name else 'Recognition',
            "id": f'recognition_{id}',
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}",
                "fps": 25
            },
            "intensityInput": {
                "mode": "binary"
            },
            "instructions": recognition_instructions,
            "instructionsType": instructions_type,
            "required": True,
            "maxSubmissions": max_sub,
            "autoSubmit": True,
            "showCountdown": True
      },
      {
            "type": "QuestionnaireTask",
            "name": f'{name}: Rating' if name else 'Rating',
            "id": f'rating_{id}',
            "form": {
            "fields": [
                {
                    "name": "laughter",
                    "label": "Does laughter occur?",
                    "input": {
                        "inputType": "Switch",
                        "defaultChecked": True,
                        "checkedChildren": "Yes",
                        "unCheckedChildren": "No"
                    }
                },
                {
                  "name": "intensity",
                  "label": "How intense was the laughter?",
                  "condition": "laughter",
                  "input": {
                    "inputType": "Slider",
                    "min": 1,
                    "max": 7,
                    "defaultValue": 4,
                    "marks": {
                        "1": "1: Almost imperceptible",
                        "2": "2",
                        "3": "3",
                        "4": "4",
                        "5": "5",
                        "6": "6",
                        "7": "7: Extremely intense"
                    },
                    "dots": True,
                    "included": False
                  }
                },
                {
                  "name": "confidence",
                  "label": "How confident are you that laughter did (not) occur?",
                  "input": {
                    "inputType": "Slider",
                    "min": 1,
                    "max": 7,
                    "defaultValue": 4,
                    "marks": {
                        "1": "1: Highly uncertain",
                        "2": "2",
                        "3": "3",
                        "4": "4",
                        "5": "5",
                        "6": "6",
                        "7": "7: Highly confident"
                    },
                    "dots": True,
                    "included": False
                  }
                }
            ]
            },
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}",
            },
            "instructions": rating_instructions,
            "instructionsType": instructions_type,
            "required": True,
            "maxSubmissions": 1
      }]

In [4]:
def get_reaction_time_task(condition, block, fpath):

    if condition == 'video':
        instructions = 'This task is designed to measure your reaction time. Please play the video and press \"q\" when a circle appears in the video. Try to keep \"q\" pressed while the circle is on and not pressed while the circle is off. Some delay is normal.'
    # if condition == 'audio':
    #     instructions = 'This task is designed to measure your reaction time. Please play the video and press \"q\" when you hear a beep. Try to keep \"q\" pressed while the beep is on and not pressed while the circle is off. Some delay is normal.'
    # if condition == 'av':
    #     instructions = 'This task is designed to measure your reaction time. Please play the video and press \"q\" when the circle appears in the video. You should hear a beep at the same time. Try to keep \"q\" pressed while the circle/beep are on and not pressed while the circle/beep are off. Some delay is normal.'
    if condition == 'explainer':
        instructions = 'This task is designed to measure your reaction time. You will encounter other tasks like this one later on. Consider this one a demo task that you can use to understand how it all works: when you play the video a circle will appear in it at any time. Press \"q\" when the circle appears. Try to keep \"q\" pressed while the circle is on and not pressed while the circle is off. Some delay is normal.'

    return {
            "type": "Continuous1DTask",
            "name": f'Reaction Time',
            "id": f'reaction-time_{condition}_{block}',
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}",
                "fps": 30
            },
            "intensityInput": {
                "mode": "binary"
            },
            "instructions": ' ' + instructions,
            "instructionsType": 'popped',
            "required": True,
            "maxSubmissions": 10 if condition == 'explainer' else 1,
            "autoSubmit": True,
            "showCountdown": True
      }

In [5]:
def get_rating_tasks(ex, idx, condition, block, fpath, calibration=False):
    if calibration:
      recognition_instructions = 'This is a special calibration task where you know that the person indicated in the video will laugh. Please play the video and press \"q\" as soon as the person starts laughing, and release it when the person stops laughing. Do not worry about pressing or releasing the key too late. Some delay is normal.'
      rating_instructions = 'Now please rate how intense you perceived the laughter to be. The video is the same as in the previous task and you may replay it.'
    else:
      recognition_instructions = 'Please play the video and try to keep \"q\" pressed when the person is laughing and not pressed when the person is not laughing. Do not worry about pressing or releasing the key too late. Some delay is normal.'
      rating_instructions = 'Now please give the required ratings on the right. You may replay the video.'
    return get_segment_tasks(
      fpath=f"{fpath}/{ex.get_file_id()}.mp4",
      name='Calibration' if calibration else None,
      id=f'{ex.get_hash()}_{condition}_{block}_{1 if calibration else 0}',
      recognition_instructions=recognition_instructions,
      rating_instructions=rating_instructions,
      instructions_type='popped' if (calibration)  else 'default',
      
    )

In [6]:
def get_example_tasks():
    recognition_instructions = 'Please play the video and try to keep \"q\" pressed when the person is laughing and not pressed when the person is not laughing. Do not worry about pressing or releasing the key too late. Some delay is normal.'
    rating_instructions = 'Now please give the required ratings on the right. You may replay the video.'

    fpaths = [
        'samples/01_793aeaa0ddbbceb35fee31c08b107b9b6ce34436c6bdfa673a601aa4f1681825_cam_1.mp4',
        'samples/17_4cb60a84e446de7d0077be4912fe9931abe6bec2e3777c28c9bf2ae8b9358b01_cam_1.mp4',
        'samples/25_a460e46f02b6c58e8d485f7a970cdc24bff1995f52a1e8040f08356ecde8cb35_cam_1.mp4'
    ]

    tasks = [get_segment_tasks(
      fpath=fpaths[i],
      name=f"Example #{i+1}",
      id=f'example{i}',
      recognition_instructions=recognition_instructions,
      rating_instructions=rating_instructions,
      instructions_type='popped' if i == 0 else 'default',
      max_sub=10
    ) for i in range(3)]

    return [t for arr in tasks for t in arr]

In [7]:
def get_hit(name, s1, s2, s3, calibration_set):

    rt_explainer = get_reaction_time_task('explainer', 0, 'artificial_examples/example2_video.mp4')

    video_calibration = [
        get_rating_tasks(calibration_set[0], 0, 'video', 1, fpath='calibration_examples/video', calibration=True),
        get_rating_tasks(calibration_set[1], 0, 'video', 1, fpath='calibration_examples/video', calibration=True)
    ]
    s1_tasks = [get_rating_tasks(ex[1], i+1, 'video', 1, fpath=f'{ex[0]}_examples/video') for i, ex in enumerate(s1)]
    s1_rtt = get_reaction_time_task('video', 1, 'artificial_examples/example1_video.mp4')


    audio_calibration = [
        get_rating_tasks(calibration_set[2], 0, 'audio', 2, fpath='calibration_examples/aiv', calibration=True),
        get_rating_tasks(calibration_set[3], 0, 'audio', 2, fpath='calibration_examples/aiv', calibration=True)
    ]
    s2_tasks = [get_rating_tasks(ex[1], i+1, 'audio', 2, fpath=f'{ex[0]}_examples/aiv') for i, ex in enumerate(s2)]
    s2_rtt = get_reaction_time_task('video', 2, 'artificial_examples/example3_video.mp4')

    av_calibration = [
        get_rating_tasks(calibration_set[4], 0, 'av', 3, fpath='calibration_examples/av', calibration=True),
        get_rating_tasks(calibration_set[5], 0, 'av', 3, fpath='calibration_examples/av', calibration=True)
    ]
    s3_tasks = [get_rating_tasks(ex[1], i+1, 'av', 3, fpath=f'{ex[0]}_examples/av') for i, ex in enumerate(s3)]
    s3_rtt = get_reaction_time_task('video', 3, 'artificial_examples/example5_video.mp4')

    return {
        "id": name,
        "name": name,
        "interface": {
            "type": "timeline"
        },
        "tasks": [
            get_consent_task(),
            get_general_instructions_task(),
            rt_explainer,
            *get_example_tasks(),

            {
                "type": "shuffle",
                "tasks": [
                    # video block
                    [ 
                        get_condition_instructions_task('video_instructions.md', 'video'),
                        *video_calibration[0],
                        {
                            "type": 'shuffle',
                            "tasks": s1_tasks
                        },
                        *video_calibration[1],
                        s1_rtt
                    ],

                    # audio block
                    [ 
                        get_condition_instructions_task('audio_instructions.md', 'audio'),
                        *audio_calibration[0],
                        {
                            "type": 'shuffle',
                            "tasks": s2_tasks
                        },
                        *audio_calibration[1],
                        s2_rtt
                    ],

                    # av block
                    [ 
                        get_condition_instructions_task('av_instructions.md', 'av'),
                        *av_calibration[0],
                        {
                            "type": 'shuffle',
                            "tasks": s3_tasks
                        },
                        *av_calibration[1],
                        s3_rtt
                    ],
                ]
            },
            
            get_experience_rating()
        ]
    }

In [8]:
import json
import os
import random
import numpy as np
import pandas as pd
sys.path.append('../lared')
from dataset.example import VideoExample, AudioExample, FullExample
random.seed(22)
np.random.seed(22)

In [9]:
pilot_path = '/home/jose/drive/data/lared_laughter/laughter_data'
laughter_examples_df = pd.read_csv(os.path.join(pilot_path, 'laughter_examples', 'examples.csv'), index_col=0)
laughter_examples_df = laughter_examples_df[laughter_examples_df['rect'].notnull()]
num_laughter_examples = len(laughter_examples_df)
laughter_examples_df = laughter_examples_df[laughter_examples_df['rect'] != '[]']
num_laughter_examples_df_bb = len(laughter_examples_df)
laughter_examples_df = laughter_examples_df.drop_duplicates(subset = ['pid', '_ini_time', '_end_time'])
print(f'LAUGHTER: Total examples: {num_laughter_examples}, With BB: {num_laughter_examples_df_bb}, Without duplicates: {len(laughter_examples_df)}')

speech_examples_df = pd.read_csv(os.path.join(pilot_path, 'speech_examples', 'examples.csv'), index_col=0)
speech_examples_df = speech_examples_df[speech_examples_df['rect'].notnull()]
num_speech_examples = len(speech_examples_df)
speech_examples_df = speech_examples_df[speech_examples_df['rect'] != '[]']
num_speech_examples_df_bb = len(speech_examples_df)
speech_examples_df = speech_examples_df.drop_duplicates(subset = ['pid', '_ini_time', '_end_time'])
print(f'SPEECH: Total examples: {num_speech_examples}, With BB: {num_speech_examples_df_bb}, Without duplicates: {len(speech_examples_df)}')

LAUGHTER: Total examples: 1000, With BB: 704, Without duplicates: 459
SPEECH: Total examples: 324, With BB: 324, Without duplicates: 169


## Create the example objects

In [10]:
laughter_examples = [('laughter', FullExample(**ex[1].to_dict())) for ex in laughter_examples_df.iterrows()]
speech_examples = [('speech', FullExample(**ex[1].to_dict())) for ex in speech_examples_df.iterrows()]

In [11]:
laughter_examples_df[laughter_examples_df['cam'] == 3].head(20)

,id,pid,cam,valid,hash,_ini_time,_end_time,ini_time,end_time,labels,rect,ini,len
1,NaN,1,3,True,adbd2fbe228c7178772c306b43ef47ffde64f6670b66b1...,1026.363079,1032.543362,1028.280,1030.080,"{'vad': None, 'label': None, 'vad_seg': None}","[159, 476, 177, 237]",NaN,NaN
10,NaN,1,3,True,10608dcffec9ca91518034c93d8207e9714ff86179af03...,1138.238934,1144.962083,1140.280,1142.080,"{'vad': None, 'label': None, 'vad_seg': None}","[126, 520, 186, 261]",NaN,NaN
13,NaN,1,3,True,fc95b15d7a78e4037d8a4c78d6bc0c3e0b4aa3ba169cad...,1165.079191,1171.823902,1167.020,1168.700,"{'vad': None, 'label': None, 'vad_seg': None}","[71, 473, 247, 231]",NaN,NaN
22,NaN,1,3,True,e739e2f626b7823cc52ced2e6e0f70e3fea55a4ac4291a...,1475.400000,1479.052282,1476.540,1477.540,"{'vad': None, 'label': None, 'vad_seg': None}","[87, 626, 259, 256]",NaN,NaN
25,NaN,1,3,True,98ad7346324dad0cbd9d9dc641618456aab2a307904d56...,1520.835912,1527.695664,1523.880,1524.280,"{'vad': None, 'label': None, 'vad_seg': None}","[84, 628, 283, 269]",NaN,NaN
65,NaN,1,3,True,01d645b5651160faf98945b1676cd34288b343936a6c8c...,4572.186668,4579.277081,4575.160,4576.120,"{'vad': None, 'label': None, 'vad_seg': None}","[343, 83, 152, 211]",NaN,NaN
105,NaN,1,3,True,2009767985be50548bb2ebeac38b7b2947c4daa2bd8410...,5776.367003,5783.363671,5779.640,5781.360,"{'vad': None, 'label': None, 'vad_seg': None}","[220, 12, 129, 151]",NaN,NaN
185,NaN,2,3,True,b46a7366c75b36280a6f1c83824a8a3cb9aeb772fd3000...,5701.614267,5712.324213,5705.076,5709.209,"{'vad': None, 'label': None, 'vad_seg': None}","[689, 161, 161, 265]",NaN,NaN
197,NaN,2,3,True,2701dc1e67e6101890bc9290138c357436ac19fcb5c924...,6063.220734,6068.649575,6064.918,6066.768,"{'vad': None, 'label': None, 'vad_seg': None}","[697, 208, 160, 196]",NaN,NaN
209,NaN,3,3,True,05d438e7b2ba50a32a1a2c99e8ce66abcec6bd1529e4bb...,1362.897371,1368.805563,1365.760,1366.420,"{'vad': None, 'label': None, 'vad_seg': None}","[738, 56, 147, 209]",NaN,NaN


In [12]:
pilot_path = '/home/jose/drive/data/lared_laughter/pilot2'
pilot_examples_df = pd.read_csv(os.path.join(pilot_path, 'laughter_examples', 'examples.csv'), index_col=0)
pilot_examples = [FullExample(**ex[1].to_dict()) for ex in pilot_examples_df.iterrows()]

In [13]:
obvious_examples_ids = [
    '64a92aea9395ace7ac9d60eab34911e419fc66610cd76f1e29df4b4fd16f230f',
    'ced6e78fe7940c10fbc9d7c385273e68459ca399ccb668c8123cf5a66fa99819',
    'bb6337eea970487ce9cd4ff26ea78c7acc6d5d1a355b7aa50029a3229f115b21',
    '85aac70ec91eb3be1b313b33e0b7828394bbe4e4edc6a956d1e7061dfc8b250e',
    '68d229cf19eec82f37580265ea93892117dd5b559b04d22489da3593315f18e7',
    'b438e94f7fdcc80ea7927e320946c7b407b7c850fe04baa60a07a5df7d92a711'
]
obvious_examples = [ex for ex in pilot_examples if ex.get_hash() in obvious_examples_ids]
len(pilot_examples), len(obvious_examples)

(2191, 6)

In [15]:
examples_dicts = [ex.to_dict() for ex in obvious_examples]
pd.DataFrame(examples_dicts).to_csv(os.path.join('/home/jose/drive/data/lared_laughter', 'laughter_data/calibration_examples.csv'))

# HIT group creation

- HIT groups have 84 examples: 25 speech + 59 laughter

In [14]:
def make_hitgroup(i, examples, duh_examples):
    set1 = examples[0:28]
    set2 = examples[28:56]
    set3 = examples[56:]

    return [
        get_hit(f'G{i}_HIT1', set1, set2, set3, duh_examples),
        get_hit(f'G{i}_HIT2', set2, set3, set1, duh_examples),
        get_hit(f'G{i}_HIT3', set3, set1, set2, duh_examples)
    ]

In [15]:
num_hitgroups = min(len(laughter_examples) // 59, len(speech_examples) // 25)
num_hitgroups

6

In [16]:
random.shuffle(laughter_examples)
random.shuffle(speech_examples)

In [17]:
hits = []
for i in range(num_hitgroups):
    lex = laughter_examples[i*59:(i+1)*59]
    sex = speech_examples[i*25:(i+1)*25]
    
    examples = [*lex, *sex]
    random.shuffle(examples)
    hits = [*hits, *make_hitgroup(i, examples, obvious_examples)]

In [18]:
project = {
    "name": "Laughter #1",
    "id": "laughter_1",
    "email": "j.d.vargasquiros@tudelft.nl",
    "hits": hits
}

In [19]:
json.dump(project, open('laughter1.covfee.json', 'w'), indent=2)